In [37]:
# import altair with an abbreviated alias
import altair as alt
# load a sample dataset as a pandas DataFrame
from vega_datasets import data

import pandas as pd
import numpy as np


In [38]:
measurement_sizes = {
    'BMP390 Temperature': 4,
    'BMP390 Pressure': 4,
    'LSM6DSO Acc': 4*3,
    'LSM6DSO Gyro': 4*3,
    'MS5611 Press': 4,
    'MS5611 Temp': 4,
}
groups = {
    'MS5611': ['MS5611 Press', 'MS5611 Temp'],
    'LSM6DSO': ['LSM6DSO Acc', 'LSM6DSO Gyro'],
    'BMP390': ['BMP390 Temperature', 'BMP390 Pressure'],
}

# ms
stages = [
    ['boost', 3],
    ['ascent', 30],
    ['flight', 302],
]

stage_rates = {
    'boost': {'MS5611': 1024, 'LSM6DSO': 416, 'BMP390': 50},
    'ascent': {'MS5611': 1024, 'LSM6DSO': 208, 'BMP390': 180},
    'flight': {'MS5611': 512, 'LSM6DSO': 52, 'BMP390': 50},
}

timestamp_size_bytes = 4

flash_chip_size_bytes = 128 * 1024 * 1024

sizedf = pd.DataFrame.from_dict(
    measurement_sizes, orient='index', columns=['size_bytes'])

# stageratedf = pd.DataFrame(.T.stack(), columns=['rate_hz']).unstack()
stageratedf = pd.DataFrame.from_dict(stage_rates).T

stagelendf = pd.DataFrame.from_records(
    stages, columns=['stage', 'time_s'], index='stage')

packetsizedf = pd.DataFrame.from_records([(groupname, sum([measurement_sizes[sensor] for sensor in groups[groupname]]))
                                         for groupname in groups.keys()], columns=['group', 'data'], index='group')
packetsizedf['timestamp'] = timestamp_size_bytes


In [39]:
combined = stageratedf.stack().to_frame(name='read_rate')
combined.index.set_names(['stage', 'group'], inplace=True)

totals = combined.join(packetsizedf) \
    .apply(lambda r: [r['read_rate'] * r['data'], r['read_rate'] * r['timestamp']], axis=1, result_type='expand') \
    .rename(columns={0: 'data', 1: 'timestamp'}).stack().rename('rate').rename_axis(['stage', 'group', 'part']).to_frame()

rates = combined.join(packetsizedf) \
    .apply(lambda r: [r['read_rate'] * r['data'], r['read_rate'] * r['timestamp']], axis=1, result_type='expand') \
    .rename(columns={0: 'data', 1: 'timestamp'}).stack().rename('rate').rename_axis(['stage', 'group', 'part']).to_frame()


In [40]:
packetsizedf['size_with_ts'] = packetsizedf['data']  + packetsizedf['timestamp']

In [41]:
combined


read_rate
stage  group             
boost  MS5611        1024
       LSM6DSO        416
       BMP390          50
ascent MS5611        1024
       LSM6DSO        208
       BMP390         180
flight MS5611         512
       LSM6DSO         52
       BMP390          50

In [42]:
combined['data_byte_rate'] = combined['read_rate'].mul(
    packetsizedf['data'], level=1)
combined['timestamp_byte_rate'] = combined['read_rate'].mul(
    timestamp_size_bytes)
combined['packet_byte_rate'] = combined['read_rate'].mul(
    packetsizedf['size_with_ts'], level=1)


In [43]:
combined['num_samples'] = combined['read_rate'].mul(
    stagelendf['time_s']).astype(int)
combined['data_total_bytes'] = combined['num_samples'].mul(
    packetsizedf['size_with_ts'], level=1).astype(int)
combined['timestamp_total_bytes'] = combined['num_samples'].mul(
    timestamp_size_bytes).astype(int)
combined['packet_total_bytes'] = combined['num_samples'].mul(
    packetsizedf['size_with_ts'], level=1).astype(int)


In [44]:

totals = combined.sum().to_frame(
    'total').T[['data_total_bytes', 'timestamp_total_bytes', 'packet_total_bytes']]
totals


,data_total_bytes,timestamp_total_bytes,packet_total_bytes
total,3158168,929032,3158168


In [45]:
def littlefs_overhead(length):
    return length * 1.01 + 2048


In [46]:
totals.map(littlefs_overhead).rename({'total': 'including_overhead'})


,data_total_bytes,timestamp_total_bytes,packet_total_bytes
including_overhead,3191797.68,940370.32,3191797.68


In [47]:
combined


read_rate  data_byte_rate  timestamp_byte_rate  \
stage  group                                                     
boost  MS5611        1024            8192                 4096   
       LSM6DSO        416            9984                 1664   
       BMP390          50             400                  200   
ascent MS5611        1024            8192                 4096   
       LSM6DSO        208            4992                  832   
       BMP390         180            1440                  720   
flight MS5611         512            4096                 2048   
       LSM6DSO         52            1248                  208   
       BMP390          50             400                  200   

                packet_byte_rate  num_samples  data_total_bytes  \
stage  group                                                      
boost  MS5611              12288         3072             36864   
       LSM6DSO             11648         1248             34944   
       BMP390                600          150              1800   
ascent MS5611              12288        30720            368640   
       LSM6DSO              5824         6240            174720   
       BMP390               2160         5400             64800   
flight MS5611               6144       154624           1855488   
       LSM6DSO              1456        15704            439712   
       BMP390                600        15100            181200   

                timestamp_total_bytes  packet_total_bytes  
stage  group                                               
boost  MS5611                   12288               36864  
       LSM6DSO                   4992               34944  
       BMP390                     600                1800  
ascent MS5611                  122880              368640  
       LSM6DSO                  24960              174720  
       BMP390                   21600               64800  
flight MS5611                  618496             1855488  
       LSM6DSO                  62816              439712  
       BMP390                   60400              181200

In [48]:
forDisplay = combined.join(stagelendf).rename(columns={
    'read_rate': 'Sampling Rate (hz)',
    'data_byte_rate': "Data Rate (b/s)",
    'timestamp_byte_rate': "Timestamp Rate (b/s)",
    'packet_byte_rate': "Packet Rate (b/s)",
    'num_samples': 'Total Samples',
    'data_total_bytes': 'Total Sensor Data (b)',
    'timestamp_total_bytes': 'Total Timestamp Data (b)',
    'packet_total_bytes': 'Total Data (b)',
    'time_s': 'Stage Time'
})

forDisplay.set_index(forDisplay.index.map(lambda r: (
    f'{r[0]} ({stagelendf.loc[r[0]]["time_s"]}s)', r[1]))).drop(columns=['Stage Time'])


Sampling Rate (hz)  Data Rate (b/s)  \
stage         group                                          
boost (3s)    MS5611                 1024             8192   
              LSM6DSO                 416             9984   
              BMP390                   50              400   
ascent (30s)  MS5611                 1024             8192   
              LSM6DSO                 208             4992   
              BMP390                  180             1440   
flight (302s) MS5611                  512             4096   
              LSM6DSO                  52             1248   
              BMP390                   50              400   

                       Timestamp Rate (b/s)  Packet Rate (b/s)  Total Samples  \
stage         group                                                             
boost (3s)    MS5611                   4096              12288           3072   
              LSM6DSO                  1664              11648           1248   
              BMP390                    200                600            150   
ascent (30s)  MS5611                   4096              12288          30720   
              LSM6DSO                   832               5824           6240   
              BMP390                    720               2160           5400   
flight (302s) MS5611                   2048               6144         154624   
              LSM6DSO                   208               1456          15704   
              BMP390                    200                600          15100   

                       Total Sensor Data (b)  Total Timestamp Data (b)  \
stage         group                                                      
boost (3s)    MS5611                   36864                     12288   
              LSM6DSO                  34944                      4992   
              BMP390                    1800                       600   
ascent (30s)  MS5611                  368640                    122880   
              LSM6DSO                 174720                     24960   
              BMP390                   64800                     21600   
flight (302s) MS5611                 1855488                    618496   
              LSM6DSO                 439712                     62816   
              BMP390                  181200                     60400   

                       Total Data (b)  
stage         group                    
boost (3s)    MS5611            36864  
              LSM6DSO           34944  
              BMP390             1800  
ascent (30s)  MS5611           368640  
              LSM6DSO          174720  
              BMP390            64800  
flight (302s) MS5611          1855488  
              LSM6DSO          439712  
              BMP390           181200

In [49]:
totals = combined[['data_total_bytes', 'timestamp_total_bytes', 'packet_total_bytes']] \
    .rename(columns={'data_total_bytes': 'data', 'timestamp_total_bytes': 'timestamp', 'packet_total_bytes': 'packet'}) \
    .stack() \
    .rename('total') \
    .rename_axis(['stage', 'group', 'part']) \
    .to_frame()

rates = combined[['data_byte_rate', 'timestamp_byte_rate', 'packet_byte_rate']] \
    .rename(columns={'data_byte_rate': 'data', 'timestamp_byte_rate': 'timestamp', 'packet_byte_rate': 'packet'}) \
    .stack() \
    .rename('rate') \
    .rename_axis(['stage', 'group', 'part']) \
    .to_frame()


In [50]:
noind = totals.join(rates).reset_index()
parts = noind[noind['part'] != 'packet'].set_index(['stage', 'group', 'part'])


In [51]:
parts.unstack()


total            rate          
part               data timestamp  data timestamp
stage  group                                     
ascent BMP390     64800     21600  1440       720
       LSM6DSO   174720     24960  4992       832
       MS5611    368640    122880  8192      4096
boost  BMP390      1800       600   400       200
       LSM6DSO    34944      4992  9984      1664
       MS5611     36864     12288  8192      4096
flight BMP390    181200     60400   400       200
       LSM6DSO   439712     62816  1248       208
       MS5611   1855488    618496  4096      2048

In [52]:
import plotly.express as px


In [53]:
fig = px.sunburst(parts.reset_index(), path=[
                  'stage', 'group', 'part'], values='total', title="Data Usage")
fig.show()
